# Eigenvalue Problems
* Schur factorization
* Reduction to Hessenberg/tri-diagonal form (bidiagonal form for SVD)
* Power method
* Inverse iteration
* Rayleigh quotient iteration
* Other iteration methods (QR algorithm, divide and conquer, Jacobi method)

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from numpy import *
from numpy.linalg import *

## Overview

There are a few obvious algorithms for finding eigenvalues, but they are bad. One example is to compute the roots of the characteristic polynomial, but polynomial root finding is an ill-conditioned problem.

Another method that we will discuss is power iteration, which is straightforward but generally slow.

The best eigenvalue algorithms rely on computing factorizations of $A$ which reveal the eigenvalues of $A$. For instance, diagonalization gives a list of all the eigenvalues on the diagonal of $\Lambda$. However, not all matrices are diagonalizable. The best we can do for general matrices is to compute the Schur decomposition $A = QTQ^*$, which gives a similarity transformation to an upper triangular matrix, which has its eigenvalues on the diagonal.

However, there are still some difficulties. Matrix decomposition algorithms we have discussed earlier that converge in a finite number of steps will not suffice. For instance, if we could compute the eigenvalues by finding the Schur factorization in a finite number of steps, then we'd have a method of computing the roots of polynomials in a finite number of arithmetic operations. But this is not the case, as we know that there is no analytic solution to polynomials of degree greater than 5. Therefore, we will need iterative methods which converge toward the true solution, in hopefully a smallish number of steps!

## Fundamentals
Let $A \in \mathbb{C}^{m\times m}$. We want numerical methods for finding the eigenvalues of $A$.

Any eigenvalue solver must be an iterative method, because we know from Galois theory that there is no analytical solution to the quintic, for instance. 

Some review:
* $A$ is *diagonalizable* if there is some $X$ such that $A = X\Lambda X^{-1}$ for a diagonal matrix $\Lambda$.
* The *geometric multiplicity* of an eigenvalue $\lambda_j$ is the dimension of the span of eigenvectors of $\lambda_j$.
* The *algebraic multiplicity* of $\lambda_j$ is multiplicity of the root $\lambda_j$ in the characteristic polynomial.

## Schur factorization

For any nxn matrix $A$, there is a unitary matrix $Q$ ($Q^*Q = I$) and an upper triangular matrix $T$ such that
$$A = QTQ^*.$$

Proof is possible by induction and block matrices.

If $A = A^*$, then $T = T^*$ is diagonal. But in general, $T$ has eigenvalues on the diagonal.

Can we compute $A = QTQ^*$ by a finite number of Householder transformations? Well, no, because then we'd have a method of computing all the eigenvalues from the diagonal of T in a finite number of steps, which we know isn't possible.

The way we will need to compute the Schur factorization is through an iterative procedure
$$ T_j = Q_j^* \dots Q_1^* A Q_1 \dots Q_j $$
where $T_j \to T$ as $j \to \infty$.

### Reduction to Hessenberg form: Householder tridiagonalization. 
We compute the decomposition $ T_j = Q_j^* \dots Q_1^* A Q_1 \dots Q_j $ in two phases: first, we use a direct method for reducing $A$ into a pseudo-upper triangular form $H = Q^* A Q$, where we have 0s below the first subdiagonal. Then, we use an iterative method to converge to the triangular matrix $T$. 

The point of doing this is that we can have methods that tend to be much more computationally efficient than if we just keep applying $Q_j$s, which takes $O(m^3)$ flops each time.

#### Golub Kahan
What about computing the SVD? We do a similar method, called the *Golub Kahan Bidiagonalization*: We apply a series of operations $U_{n-2}^* \dots U_1^* A V_1 \dots V_{n-2}$ to get a bidiagonal matrix $B$, from which we can apply an iterative method that will converge in just $m$ iterations.

## Power method

Assume $A = A^T$ for convenience.

Let $v^{(0)} \in \mathbb{R}^m$
for k = 1, 2, ...
$$\begin{align}
    w &= Av^{(k-1)} \\
    v^{(k)} &= w / \|w\|
\end{align}$$

What does this algorithm converge to? Let $A = Q \Lambda Q^T$, and write $v^{(0)} = a_1 q_1 + \dots a_m q_m$ where the $q_i$ are columns of $Q$. (We move the columns around so that $|\lambda_1| > |\lambda_2| \geq \dots \geq |\lambda_m|$.

Then
$$\begin{align}
v^{(k)} &= c_k A^k v^{(0)} \\
&= c_k (a_1 A^k q_1 + \dots a_m A^k q_m) \\
&= c_k (a_1 \lambda_1^k q_1 + \dots a_m \lambda_m^k q_m) \\
&= c_k \lambda_1^k \left(a_1 q_1 + a_2 \left(\frac{\lambda_2}{\lambda_1}\right)^k \dots a_m \left(\frac{\lambda_m} {\lambda_1}\right)^k q_m \right)
&\to c_k \lambda_1^k a_1 q_1,
\end{align}$$
assuming $a_1 \neq 0$.

So $v^{(k)}$ converges to a multiple of $q_1$. Since $v_k$ and $q_1$ are normalized, we get $\| v_k - (\pm q_1) \| = O \left(\left| \frac{\lambda_1}{\lambda_2}\right|^k \right)$. The implication that the power method converges to the principal eigenvector, but may be slow if $|\lambda_1| \approx |\lambda_2|$.

## The Rayleigh Quotient

The Rayleigh Quotient is defined as $r(x) = \frac{x^T A x}{xTx}$, for $x \neq 0$.

If $x$ is an approximation of an eigenvector then the Rayleigh quotient should approximate the eigenvalue.

Assume that $A = A^T$ for some nice and useful properties. 

Properties:
* $r(x)$ is the solution of the modified least squares problem $\min_{\alpha \in \mathbb{R}} \|Ax - \alpha x\|$
* $\frac{\partial r}{\partial x_j} = \frac{2}{x^Tx}(Ax - r(x)x)_j $ so $\nabla r(x) = \frac{2}{x^Tx} (Ax - r(x)x)$. So $\nabla r(x) = 0 \iff r(x)$ is an eigenvector of $A$ with eigenvector $x$.
* Suppose $\|x - q_j\| = \epsilon$ where $q_j$ is an eigenvector. Then we can expand via the Taylor series:
$$\begin{align}
r(x) &= r(q_j) + \nabla r(q_j)^T(x - q_j) + \frac{1}{2} (x - q_j)^T \nabla^2 r(q_j)^T(x - q_j) \\
&= \lambda_j + O(\epsilon^2)
\end{align}$$

Return to power iteration: we know $\| v_k - (\pm q_1) \| = O \left(\left| \frac{\lambda_1}{\lambda_2}\right|^k \right)$,

so $|\lambda^{(k)} - \lambda_1 | = \| v_k - (\pm q_1) \| = O \left(\left| \frac{\lambda_1}{\lambda_2}\right|^k \right)$.

## Inverse power iteration

Consider $\mu \in \mathbb{R}$ which is not an eigenvalue $\lambda_j$ of $A$. Then
$$\begin{align}
A - \mu I &= Q (\Lambda - \mu I) Q^T \\
(A - \mu I)^{-1} &= Q (\Lambda - \mu I)^{-1} Q^T \\
\end{align}$$

where $(\Lambda - \mu I)^{-1}$ is a diagonal matrix and has eigenvalues $\frac{1}{\lambda_j - \mu}$.

So if we choose $\mu$ close to some $\lambda_J$, then $| \mu - \lambda_J| < |\mu - \lambda_K| \leq | \mu - \lambda_j|$, and apply the power method, we can get much faster convergence:

**Algorithm** (Inverse iteration).

for k=1, 2, ...
* Solve $(A - \mu I)w = v_{k-1}$
* $v_k = w/\|w\|$
* $\lambda^{(k)} = v_k^T A v_k$

### Summary of results so far:
* Given $q$, we can 